# Generating Master Boot Image with a Signature Provider for MIMXRT595S

This notebook provides a guide on utilizing a custom remote signing service to generate a Master Boot Image (MBI) using the *nxpimage* tool. We will explore the integration of the signature provider with the SPSDK framework and efficient signing of your images

## 1. Prerequisites
- SPSDK is needed with examples extension. `pip install spsdk[examples]` (Please refer to the [installation](../../_knowledge_base/installation_guide.rst) documentation.)
- Connect the mimxrt595s-evk board via your preferred interface

  <img src="../../_data/img/boards/mimxrt595s-evk.png" alt="mimxrt595s-evk" height="300">

Let's prepare also workspace and variables.

In [11]:
from spsdk.utils.jupyter_utils import YamlDiffWidget

# This env variable sets colored logger output to STDOUT
%env JUPYTER_SPSDK=1
# Set a magic for command execution and echo
%alias execute echo %l && %l
%alias_magic ! execute

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


In [12]:
import os

DATA_DIR = "_data/"
WORKSPACE = "workspace/"  # change this to path to your workspace
PLUGINS_DIR = "../_common/plugins/"
SASP_PLUGIN = os.path.join(PLUGINS_DIR, "sasp.py")
FAMILY = "mimxrt595s"
VERBOSITY = (
    "-v"  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)

## 2. Signature Provider Plugin Setup

First, we need to setup the Signature Provider plugin and start the custom HSM. In order to do that, go to [Signature Provider notebook](../_common/signature_provider.ipynb) and follow the instructions there. 
Once you are done, come back and continue here.

## 3. Preparing the configuration file

To successfully create a Master Boot Image, a configuration file is required for the *nxpimage* application.
There are three types of MBIs for the RT5xx series, categorized by their authentication type: `Plain`, `CRC`, and `Signed`. In this example, we will focus exclusively on the `Signed` image type.

Let's begin with creating a template configuration file running the `nxpimage mbi get-templates` command. This command generates a YAML template that can be customized so the custom Signature Provider is integrated. Below, we'll compare the differences between the template and our customized example to highlight the additions we've made.

In [16]:
# Get difference of template and user YAML configuration
YamlDiffWidget("mimxrt595s_mbi.diffc").html

nxpimage mbi get-templates -f mimxrt595s -o workspace/ --force 
Creating workspace/mimxrt595s_xip_plain.yaml template file.
Creating workspace/mimxrt595s_xip_crc.yaml template file.
Creating workspace/mimxrt595s_xip_signed.yaml template file.
Creating workspace/mimxrt595s_load_to_ram_plain.yaml template file.
Creating workspace/mimxrt595s_load_to_ram_crc.yaml template file.
Creating workspace/mimxrt595s_load_to_ram_signed.yaml template file.
Creating workspace/mimxrt595s_load_to_ram_encrypted.yaml template file.


### 3.1 Signature Provider configuration

The signature provider configuration string must follow the format:

  ```
  "type=<identifier>;<key1>=<value1>;<key2>=<value2>;..."
  ```
 - The `<identifier>` must match the `identifier` class attribute defined in the custom Signature Provider (in this example `plugins/sasp.py`).
 - The remaining key-value pairs will be passed to the `__init__` method of the given Signature Provider.


For instance, the configuration string:
  ```
  "type=file;file_path=private_key.pem"
  ```  
  will instantiate the following object:  
  ```python
  spsdk.crypto.PlainFileSP(file_path='private_key.pem')
  ```

## 4. Master Boot Image Generation

At this point, we have everything we need to run *nxpimage* application using remote HSM for image signing.

In [15]:
CONFIG_PATH = os.path.join(DATA_DIR, "mimxrt595s_xip_signed.yaml")
OUTPUT_PATH = os.path.join(WORKSPACE, "my_mbi.bin")
%! nxpimage $VERBOSITY mbi export --plugin $SASP_PLUGIN --config $CONFIG_PATH

# check if the signed image exists
assert os.path.exists(OUTPUT_PATH)

nxpimage -v mbi export --plugin ../_common/plugins/sasp.py --config _data/mimxrt595s_xip_signed.yaml 
RKTH: db31d46c717711a8231cbc38b1de8a6e8657e1f733e04c2ee4b62fcea59149fa
INFO:spsdk.apps.nxpimage:
+==0x0000_0000= Application Block ======+
|       Size: 14.5 kiB; 14,804 B        |
|+==0x0000_0000= Application ==========+|
||      Size: 13.0 kiB; 13,264 B       ||
|+==0x0000_33cf========================+|
|+==0x0000_33d0= Certification Block ==+|
||       Size: 1.3 kiB; 1,284 B        ||
|+==0x0000_38d3========================+|
|+==0x0000_38d4= RSA signature ========+|
||             Size: 256 B             ||
|+==0x0000_39d3========================+|
+==0x0000_39d3==========================+

Success. (Master Boot Image: workspace/my_mbi.bin created.)


## 5. HSM teardown

Last step is to stop custom HSM. In order to do that, open again the [HSM Setup notebook](../_common/sahsm.ipynb) and stop the running jupyter notebook code cell.